In [6]:
import numpy as np
import pandas as pd
import random

# разделение выборки на трейн и тест
def trainTestSplit(dataFrame, testSize):
    # если testSize - флоат
    if isinstance(testSize, float):
        # получаем целое количество строк, которое будет принадлежать тестовой выборке
        testSize = round(testSize * len(dataFrame))
    # получаем лист с индексами датафрейма
    indices = dataFrame.index.tolist()
    # выбираем рандомные индексы для тестовой выборки
    testIndices = random.sample(population = indices, k = testSize)
    # в тестовую выборку забираем строки с полученными индексами
    dataFrameTest = dataFrame.loc[testIndices]
    # обучающая выборка - все остальное
    dataFrameTrain = dataFrame.drop(testIndices)
    # возвращаем обе выборки
    return dataFrameTrain, dataFrameTest

# функция классифицирования даты
def classifyData(data):
    # получаем уникальные классы и количество их представителей
    uniqueClasses, uniqueClassesCounts = np.unique(data[:, -1], return_counts = True)
    # возвращает класс с наибольшим количеством представителей - самый популярный класс
    return uniqueClasses[uniqueClassesCounts.argmax()]

# получение потенциального разделения
def getPotentialSplits(data, randomAttributes):
    # объявление пустой переменной
    potentialSplits = {}
    # получаем количество столбцов
    _, columns = data.shape
    # получаем индексы столбцов
    columnsIndices = list(range(columns - 1))
    # если 
    if randomAttributes != None and len(randomAttributes) <= len(columnsIndices):
        columnsIndices = randomAttributes
    for column in columnsIndices:
        values = data[:, column]
        uniqueValues = np.unique(values)
        if len(uniqueValues) == 1:
            potentialSplits[column] = uniqueValues
        else:
            potentialSplits[column] = []
            for i in range(len(uniqueValues)):
                if i != 0:
                    currentValue = uniqueValues[i]
                    previousValue = uniqueValues[i - 1]
                    potentialSplits[column].append((currentValue + previousValue) / 2)
    return potentialSplits

# функция разделения даты по столбцу и по его значению
def splitData(data, splitColumn, splitValue):
    # получаем все значения признака, по которому будем делить
    splitColumnValues = data[:, splitColumn]
    # возвращаем два сета - левый, где значения меньше или равны порога, правый - все остальные
    return data[splitColumnValues <= splitValue], data[splitColumnValues > splitValue]

# вычисление энтропии
def entropy(data):
    # подсчитываем элементы, принадлежащие каждому классу
    _, number_of_class = np.unique(data[:, -1], return_counts = True)
    # вычисляем вероятность каждого класса = (его представители / все представители)
    Pi = number_of_class / number_of_class.sum()
    return sum(Pi * -np.log2(Pi))

# вычисление общей энтропии
def calculateOverallEntropy(dataBelow, dataAbove):
    # вычисление энтропии снизу
    pDataBelow = len(dataBelow) / (len(dataBelow) + len(dataAbove))
    # вычисление энтропии сверху
    pDataAbove = len(dataAbove) / (len(dataBelow) + len(dataAbove))
    # возвращает общую энтропию
    return pDataBelow * entropy(dataBelow) + pDataAbove * entropy(dataAbove)

# поиск лучшего разделения
def determineBestSplit(data, potentialSplits, randomSplits = None):
    # т.к. нужна наименьшая энтропия, но изначально указываем ее максимальную
    overallEntropy = 9999
    # задаем переменную, содержащую столбец с номером лучшего признака для разделения 
    bestSplitColumn = 0
    # задаем переменную, содержащую значение, по которому будет производиться разделение
    bestSplitValue = 0
    # если делим нерандомно
    if randomSplits == None:
        # для столбцов, по которым потенциально может пройти разделение
        for splitColumn in potentialSplits:
            # для значений этих столбцов
            for splitValue in potentialSplits[splitColumn]:
                # вычисляем временное разделение
                dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
                # считаем энтропию такого разделения
                currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
                # если энтропия получилась лучше (ее значение меньше), то запоминаем данное разделение
                if currentOverallEntropy <= overallEntropy:
                    overallEntropy = currentOverallEntropy
                    bestSplitColumn = splitColumn
                    bestSplitValue = splitValue
    # если же разделение рандомное
    else:
        # для значений рандомного разделения
        for i in range(randomSplits):
            # выбираем рандомный признак для разделения
            randomSplitColumn = random.choice(list(potentialSplits))
            # выбираем рандомное пороговое значение для разделения
            randomSplitValue = random.choice(potentialSplits[randomSplitColumn])
            # делим дату в соответствии с выбранными значениями для разделения
            dataBelow, dataAbove = splitData(data, randomSplitColumn, randomSplitValue)
            # считаем энтропию
            currentOverallEntropy = calculateOverallEntropy(dataBelow, dataAbove)
            # проверяем, получился ли лучший результат, чем был найден до этого
            if currentOverallEntropy <= overallEntropy:
                overallEntropy = currentOverallEntropy
                bestSplitColumn = randomSplitColumn
                bestSplitValue = randomSplitValue
    # возвращает лучший признак и его лучшее значение для разделения
    return bestSplitColumn, bestSplitValue

# функция построения дерева решений
def buildDecisionTree(dataFrame, currentDepth = 0, minSampleSize = 2, maxDepth = 1000, randomAttributes = None, randomSplits = None):
    # если находимся в начале построения дерева, когда еще не было ни одного разделения
    if currentDepth == 0:
        # объявляем глобальную переменную, содержащую названия признаков
        global COLUMN_HEADERS
        COLUMN_HEADERS = dataFrame.columns
        # представляем датасет как numpy массив
        data = dataFrame.values
        # если были поданы рандомные атрибуты и если их длина меньше, чем количество признаков
        if randomAttributes != None and randomAttributes <= len(COLUMN_HEADERS) - 1:
            # случайным образом выбираем атрибуты среди признаков датасета
            randomAttributes = random.sample(population = list(range(len(COLUMN_HEADERS) - 1)), k = randomAttributes)
        # иначе ничего с рандомными атрибутами не делаем
        else:
            randomAttributes = None
    # иначе если дерево уже в процессе построения
    else:
        # объявляем датой поданый датафрейм
        data = dataFrame
    # если количество строк = 1 или длина оставшихся строк меньше семпл сайз, или достигли максимальной глубины
    # т.е. если достигли точки останова
    if (len(np.unique(data[:, -1])) == 1) or len(data) < minSampleSize or currentDepth == maxDepth:
        # классифицируем дату
        return classifyData(data)
    # иначе продолжаем построение
    else:
        # увеличиваем глубину
        currentDepth += 1
        # получаем потенциальное разбиение
        potentialSplits = getPotentialSplits(data, randomAttributes)
        # среди потенциального разбиения ищем лучшего кандидата
        splitColumn, splitValue = determineBestSplit(data, potentialSplits, randomSplits)
        # в соответствии с лучшим разбиением делим дату
        dataBelow, dataAbove = splitData(data, splitColumn, splitValue)
        # если в одной из веток ничего не оказалось, классифицируем дату
        if len(dataBelow) == 0 or len(dataAbove) == 0:
            return classifyData(data)
        # иначе и в левой, и в правой ветке есть данные
        else:
            # формируем строку вопроса - "<имя признака> <= <значения для разделения>"
            question = str(COLUMN_HEADERS[splitColumn]) + " <= " + str(splitValue)
            # формируем поддерево
            decisionSubTree = {question: []}
            # ответа да - рекурсивное построение левого потомка
            yesAnswer = buildDecisionTree(dataBelow, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            # ответ нет - рекурсивное построение правого потомка
            noAnswer = buildDecisionTree(dataAbove, currentDepth, minSampleSize, maxDepth, randomAttributes, randomSplits)
            # если слева и справа равные ветки, то возвращаем левую
            if yesAnswer == noAnswer:
                decisionSubTree = yesAnswer
            # иначе добавляем к дереву обе ветки
            else:
                decisionSubTree[question].append(yesAnswer)
                decisionSubTree[question].append(noAnswer)
            # возвращаем построенное поддерево
            return decisionSubTree

# функция спуска по дереву до нужного класса
def classifySample(sample, decisionTree):
    # если decisionTree больше не является деревом - остановка рекурсии, дошли до листа
    if not isinstance(decisionTree, dict):
        return decisionTree
    # проверяем первый поданый признак
    question = list(decisionTree.keys())[0]
    # делим дату так же, как делилось дерево - слева меньше
    attribute, value = question.split(" <= ")
    # если значение поданного атрибута лежит в левой ветке, то в качестве ответа выбираем левую ветку
    if sample[attribute] <= float(value):
        answer = decisionTree[question][0]
    # иначе выбираем правую
    else:
        answer = decisionTree[question][1]
    # продолжаем рекурсивный спуск
    return classifySample(sample, answer)

#  функция предсказания для целого датафрейма
def decisionTreePredictions(dataFrame, decisionTree):
    # предсказание - это применение функции classifySample к каждому столбцу dataFrame
    predictions = dataFrame.apply(classifySample, axis = 1, args = (decisionTree,))
    # возвращает предсказанное значение
    return predictions

# подсчет accuracy, в принципе не нужна
def calculateAccuracy(predictedResults, category):
    resultCorrect = predictedResults == category
    return resultCorrect.mean()


In [7]:
import time

dataFrame = pd.read_csv("df.csv")

dataFrame = dataFrame[dataFrame.columns.tolist()[1:] +
                      dataFrame.columns.tolist()[0: 1]]
dataFrameTrain, dataFrameTest = trainTestSplit(dataFrame, testSize=0.25)

print("Decision Tree - Breast Cancer Dataset")

i = 1
accuracyTrain = 0
while accuracyTrain < 100:
    startTime = time.time()
    decisionTree = buildDecisionTree(dataFrameTrain, maxDepth=7)
    buildingTime = time.time() - startTime
    decisionTreeTestResults = decisionTreePredictions(
        dataFrameTest, decisionTree)
    accuracyTest = calculateAccuracy(
        decisionTreeTestResults, dataFrameTest.iloc[:, -1]) * 100
    decisionTreeTrainResults = decisionTreePredictions(
        dataFrameTrain, decisionTree)
    accuracyTrain = calculateAccuracy(
        decisionTreeTrainResults, dataFrameTrain.iloc[:, -1]) * 100
    print("maxDepth = {}: ".format(i), end="")
    print("accTest = {0:.2f}%, ".format(accuracyTest), end="")
    print("accTrain = {0:.2f}%, ".format(accuracyTrain), end="")
    print("buildTime = {0:.2f}s".format(buildingTime), end="\n")
    i += 1


Decision Tree - Breast Cancer Dataset


KeyboardInterrupt: 